In [1]:
import camelot
import pandas as pd
import re
from MyCreds.mycreds import Capstone_AWS_Stuart
from sqlalchemy import create_engine
from geopy.geocoders import Nominatim
import time
import geopandas as gpd
from geoalchemy2 import Geometry
import requests

In [2]:
geolocator = Nominatim(user_agent="pschools_singapore")

def building_to_data(search_string):
  '''
  function to query onemap api for building name - adapted from pcode_to_data
  '''
  page = 1
  results = []
    
  while True:
      try:
          response = requests.get('http://developers.onemap.sg/commonapi/search?searchVal={0}&returnGeom=Y&getAddrDetails=Y&pageNum={1}'
                                  .format(search_string, page)) \
                              .json()
      except requests.exceptions.ConnectionError as e:
          print('Fetching {} failed. Retrying in 2 sec'.format(search_string))
          
          time.sleep(2)
          continue
          
      results = results + response['results']
      if response['totalNumPages'] > page:
          page = page + 1
      else:
          break
          
  return results

In [3]:
file = 'https://www.moe.gov.sg/microsites/psle-fsbb/assets/infographics/transition-to-secondary-school/indicative-psle-score-ranges-for-individual-secondary-schools.pdf'

table = camelot.read_pdf('data/school-directory-and-information/indicative-psle-score-ranges-for-individual-secondary-schools.pdf',pages='1-16',strip_text="\n")


In [4]:
for i in range(0,16):
    if i == 0:
        df = table[i].df.iloc[3:]
    else:
        df2 = table[i].df.iloc[3:]
        df = pd.concat([df,df2])

In [5]:
df.rename({0:'serial',
           1:"school",
           7:"score_range"},axis=1,inplace=True)

df=df[['serial','school','score_range']]

In [6]:
score_low = []
score_high = []

for i in df.iterrows():
    range = i[1][2]
    scores = re.findall(r'\d+',range)
    if len(scores) == 2:
        score_low.append(scores[0])
        score_high.append(scores[1])
    elif len(scores) == 4:
        score_low.append((int(scores[0])+int(scores[2]))/2)
        score_high.append((int(scores[1])+int(scores[3]))/2)
    elif len(scores) == 0:
        score_low.append(100)
        score_high.append(100)

In [7]:
df['score_low'] = score_low
df['score_high'] = score_high
df['average_score'] = (df['score_low'].astype(int)+df['score_high'].astype(int))/2
df['percentile_rank'] = df['average_score'].rank(pct=True, ascending=False)
df

,serial,school,score_range,score_low,score_high,average_score,percentile_rank
3,1,ADMIRALTY SECONDARY SCHOOL,15 - 20,15,20,17.5,0.102041
4,2,AHMAD IBRAHIM SECONDARY SCHOOL,10 - 17,10,17,13.5,0.482993
5,3,ANDERSON SECONDARY SCHOOL,4 - 10,4,10,7.0,0.870748
6,4,ANG MO KIO SECONDARY SCHOOL,11 - 16,11,16,13.5,0.482993
7,,5 ANGLICAN HIGH SCHOOL,5(D) - 12(P),5,12,8.5,0.782313
...,...,...,...,...,...,...,...
10,,143 YUHUA SECONDARY SCHOOL West,14 - 22,14,22,18.0,0.064626
11,144,YUSOF ISHAK SECONDARY SCHOOL#,-,100,100,100.0,0.006803
12,145,YUYING SECONDARY SCHOOL,8 - 22,8,22,15.0,0.363946
3,146,ZHENGHUA SECONDARY SCHOOL,12 - 20,12,20,16.0,0.231293


In [8]:
df.sort_values(by=['percentile_rank'])

,serial,school,score_range,score_low,score_high,average_score,percentile_rank
11,144,YUSOF ISHAK SECONDARY SCHOOL#,-,100,100,100.0,0.006803
7,63,JUNYUAN SECONDARY SCHOOL,17 - 22,17,22,19.5,0.013605
5,100,QUEENSTOWN SECONDARY SCHOOL,16 - 22,16,22,19.0,0.020408
11,56,GUANGYANG SECONDARY SCHOOL,16 - 21,16,21,18.5,0.037415
6,139,YIO CHU KANG SECONDARY SCHOOL,16 - 21,16,21,18.5,0.037415
...,...,...,...,...,...,...,...
10,8,ANGLO-CHINESE SCHOOL (INDEPENDENT) – INTEGRATE...,4 - 7,4,7,5.5,0.969388
8,33,CHIJ ST. NICHOLAS GIRLS' SCHOOL - INTEGRATED P...,4(D) - 7(M),4,7,5.5,0.969388
12,78,METHODIST GIRLS' SCHOOL (SECONDARY) - INTEGRAT...,4 - 6,4,6,5.0,0.993197
8,103,RAFFLES INSTITUTION – INTEGRATED PROGRAMME,4 - 6,4,6,5.0,0.993197


In [9]:
df['match_on'] = df['serial']+df['school']
df['match_on'] = df['match_on'].str.upper()
df['match_on'] = df['match_on'].str.replace(r'\d+','')
df['match_on'] = df['match_on'].str.replace('INTEGRATED PROGRAMME','')
df['match_on'] = df['match_on'].str.replace('#','')
df['match_on'] = df['match_on'].str.replace(' ','')
df['match_on'] = df['match_on'].str.replace('-','')
df['match_on'] = df['match_on'].str.replace('–','')
df['match_on'] = df['match_on'].str.replace('(','')
df['match_on'] = df['match_on'].str.replace(')','')
df['match_on'] = df['match_on'].str.replace('.','')
df['match_on'] = df['match_on'].str.replace("'",'')
df['match_on'] = df['match_on'].str.replace('MARISSTELLAHIGHSCHOOLNORTH','MARISSTELLAHIGHSCHOOL')
df['match_on'] = df['match_on'].str.replace('FUHUASECONDARYSCHOOLWEST','FUHUASECONDARYSCHOOL')
df['match_on'] = df['match_on'].str.replace('HUAYISECONDARYSCHOOLWEST','HUAYISECONDARYSCHOOL')
df['match_on'] = df['match_on'].str.replace('JUYINGSECONDARYSCHOOLWEST','JUYINGSECONDARYSCHOOL')
df['match_on'] = df['match_on'].str.replace('KRANJISECONDARYSCHOOLWEST','KRANJISECONDARYSCHOOL')
df['match_on'] = df['match_on'].str.replace('PEICAISECONDARYSCHOOLSOUTH','PEICAISECONDARYSCHOOL')
df['match_on'] = df['match_on'].str.replace('XINMINSECONDARYSCHOOLNORTH','XINMINSECONDARYSCHOOL')
df['match_on'] = df['match_on'].str.replace('YUHUASECONDARYSCHOOLWEST','YUHUASECONDARYSCHOOL')
df

/Users/stuartong/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/stuartong/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  if __name__ == '__main__':
/Users/stuartong/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  # Remove the CWD from sys.path while we load stuff.
/Users/stuartong/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11:

,serial,school,score_range,score_low,score_high,average_score,percentile_rank,match_on
3,1,ADMIRALTY SECONDARY SCHOOL,15 - 20,15,20,17.5,0.102041,ADMIRALTYSECONDARYSCHOOL
4,2,AHMAD IBRAHIM SECONDARY SCHOOL,10 - 17,10,17,13.5,0.482993,AHMADIBRAHIMSECONDARYSCHOOL
5,3,ANDERSON SECONDARY SCHOOL,4 - 10,4,10,7.0,0.870748,ANDERSONSECONDARYSCHOOL
6,4,ANG MO KIO SECONDARY SCHOOL,11 - 16,11,16,13.5,0.482993,ANGMOKIOSECONDARYSCHOOL
7,,5 ANGLICAN HIGH SCHOOL,5(D) - 12(P),5,12,8.5,0.782313,ANGLICANHIGHSCHOOL
...,...,...,...,...,...,...,...,...
10,,143 YUHUA SECONDARY SCHOOL West,14 - 22,14,22,18.0,0.064626,YUHUASECONDARYSCHOOL
11,144,YUSOF ISHAK SECONDARY SCHOOL#,-,100,100,100.0,0.006803,YUSOFISHAKSECONDARYSCHOOL
12,145,YUYING SECONDARY SCHOOL,8 - 22,8,22,15.0,0.363946,YUYINGSECONDARYSCHOOL
3,146,ZHENGHUA SECONDARY SCHOOL,12 - 20,12,20,16.0,0.231293,ZHENGHUASECONDARYSCHOOL


In [10]:
schools = 'data/school-directory-and-information/general-information-of-schools.csv'

all_schools = pd.read_csv(schools)

secondary = all_schools.copy()
# secondary = all_schools[(all_schools['mainlevel_code']=='SECONDARY') | (all_schools['mainlevel_code']=='MIXED') ]
secondary['match_on'] = secondary['school_name']
secondary['match_on'] = secondary['match_on'].str.upper()
secondary['match_on'] = secondary['match_on'].str.replace('INTEGRATED PROGRAMME','')
secondary['match_on'] = secondary['match_on'].str.replace(r'\d+','')
secondary['match_on'] = secondary['match_on'].str.replace('#','')
secondary['match_on'] = secondary['match_on'].str.replace(' ','')
secondary['match_on'] = secondary['match_on'].str.replace('-','')
secondary['match_on'] = secondary['match_on'].str.replace('(','')
secondary['match_on'] = secondary['match_on'].str.replace(')','')
secondary['match_on'] = secondary['match_on'].str.replace('.','')
secondary['match_on'] = secondary['match_on'].str.replace("'",'')



secondary

/Users/stuartong/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  # Remove the CWD from sys.path while we load stuff.
/Users/stuartong/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  
/Users/stuartong/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  from ipykernel import kernelapp as app
/Users/stuartong/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: FutureWarning: The default value of regex will change from True 

,school_name,url_address,address,postal_code,telephone_no,telephone_no_2,fax_no,fax_no_2,email_address,mrt_desc,...,session_code,mainlevel_code,sap_ind,autonomous_ind,gifted_ind,ip_ind,mothertongue1_code,mothertongue2_code,mothertongue3_code,match_on
0,ADMIRALTY PRIMARY SCHOOL,https://admiraltypri.moe.edu.sg/,11 WOODLANDS CIRCLE,738907,63620598,na,63627512,na,ADMIRALTY_PS@MOE.EDU.SG,Admiralty Station,...,FULL DAY,PRIMARY,No,No,No,No,Chinese,Malay,Tamil,ADMIRALTYPRIMARYSCHOOL
1,ADMIRALTY SECONDARY SCHOOL,http://www.admiraltysec.moe.edu.sg,31 WOODLANDS CRESCENT,737916,63651733,63654596,63652774,na,Admiralty_SS@moe.edu.sg,ADMIRALTY MRT,...,SINGLE SESSION,SECONDARY,No,No,No,No,Chinese,Malay,Tamil,ADMIRALTYSECONDARYSCHOOL
2,AHMAD IBRAHIM PRIMARY SCHOOL,http://www.ahmadibrahimpri.moe.edu.sg,10 YISHUN STREET 11,768643,67592906,na,67592927,na,aips@moe.edu.sg,Yishun,...,SINGLE SESSION,PRIMARY,No,No,No,No,Chinese,Malay,Tamil,AHMADIBRAHIMPRIMARYSCHOOL
3,AHMAD IBRAHIM SECONDARY SCHOOL,http://www.ahmadibrahimsec.moe.edu.sg,751 YISHUN AVENUE 7,768928,67585384,na,67557778,na,aiss@moe.edu.sg,"CANBERRA MRT, YISHUN MRT",...,SINGLE SESSION,SECONDARY,No,No,No,No,Chinese,Malay,Tamil,AHMADIBRAHIMSECONDARYSCHOOL
4,AI TONG SCHOOL,http://www.aitong.moe.edu.sg,100 Bright Hill Drive,579646,64547672,na,64532726,na,AITONG_SCH@MOE.EDU.SG,Bishan MRT,...,SINGLE SESSION,PRIMARY,Yes,No,No,No,Chinese,na,na,AITONGSCHOOL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
341,ZHANGDE PRIMARY SCHOOL,http://www.zhangdepri.moe.edu.sg/,51 Jalan Membina,169485,62740357,na,62763042,na,ZHANGDE_PS@MOE.EDU.SG,Tiong Bahru MRT Station,...,SINGLE SESSION,PRIMARY,No,No,No,No,Chinese,Malay,Tamil,ZHANGDEPRIMARYSCHOOL
342,ZHENGHUA PRIMARY SCHOOL,http://www.zhenghuapri.moe.edu.sg,9 Fajar Road,679002,67697478,na,67636459,na,zhenghua_ps@moe.edu.sg,LRT: Fajar Station,...,SINGLE SESSION,PRIMARY,No,No,No,No,Chinese,Malay,Tamil,ZHENGHUAPRIMARYSCHOOL
343,ZHENGHUA SECONDARY SCHOOL,http://www.zhenghuasec.moe.edu.sg,91 SENJA ROAD,677741,67639455,na,67633577,na,ZHENGHUA_SS@MOE.EDU.SG,JELAPANG LRT,...,SINGLE SESSION,SECONDARY,No,No,No,No,Chinese,Malay,Tamil,ZHENGHUASECONDARYSCHOOL
344,ZHONGHUA PRIMARY SCHOOL,http://www.zhonghuapri.moe.edu.sg,12 SERANGOON AVENUE 4,556095,62835413,na,62823882,na,zhonghua_ps@moe.edu.sg,"Ang Mo Kio MRT, Bishan MRT, Serangoon MRT",...,SINGLE SESSION,PRIMARY,No,No,No,No,Chinese,Malay,Tamil,ZHONGHUAPRIMARYSCHOOL


In [11]:
secondary[secondary['school_name'].str.contains('MARIS')]

,school_name,url_address,address,postal_code,telephone_no,telephone_no_2,fax_no,fax_no_2,email_address,mrt_desc,...,session_code,mainlevel_code,sap_ind,autonomous_ind,gifted_ind,ip_ind,mothertongue1_code,mothertongue2_code,mothertongue3_code,match_on
162,MARIS STELLA HIGH SCHOOL,http://www.marisstellahigh.moe.edu.sg,25 MOUNT VERNON ROAD,368051,62803880 (Sec),62827743 (Pri),62886905 (Sec),62827905 (Pri),msh@moe.edu.sg,BARTLEY MRT,...,SINGLE SESSION,MIXED LEVELS,Yes,Yes,No,No,Chinese,na,na,MARISSTELLAHIGHSCHOOL


In [12]:
secondary_ranked = df.merge(secondary, left_on='match_on',right_on='match_on',how='left',indicator=True)

In [13]:
secondary_ranked[secondary_ranked['_merge']!='both']

,serial,school,score_range,score_low,score_high,average_score,percentile_rank,match_on,school_name,url_address,...,session_code,mainlevel_code,sap_ind,autonomous_ind,gifted_ind,ip_ind,mothertongue1_code,mothertongue2_code,mothertongue3_code,_merge


In [14]:
df[df['school'].str.contains('MARIS')]


,serial,school,score_range,score_low,score_high,average_score,percentile_rank,match_on
7,,73 MARIS STELLA HIGH SCHOOL North,7(M) - 12,7,12,9.5,0.727891,MARISSTELLAHIGHSCHOOL


In [15]:
secondary_ranked.drop(columns=['_merge'],inplace=True)
secondary_ranked

,serial,school,score_range,score_low,score_high,average_score,percentile_rank,match_on,school_name,url_address,...,nature_code,session_code,mainlevel_code,sap_ind,autonomous_ind,gifted_ind,ip_ind,mothertongue1_code,mothertongue2_code,mothertongue3_code
0,1,ADMIRALTY SECONDARY SCHOOL,15 - 20,15,20,17.5,0.102041,ADMIRALTYSECONDARYSCHOOL,ADMIRALTY SECONDARY SCHOOL,http://www.admiraltysec.moe.edu.sg,...,CO-ED SCHOOL,SINGLE SESSION,SECONDARY,No,No,No,No,Chinese,Malay,Tamil
1,2,AHMAD IBRAHIM SECONDARY SCHOOL,10 - 17,10,17,13.5,0.482993,AHMADIBRAHIMSECONDARYSCHOOL,AHMAD IBRAHIM SECONDARY SCHOOL,http://www.ahmadibrahimsec.moe.edu.sg,...,CO-ED SCHOOL,SINGLE SESSION,SECONDARY,No,No,No,No,Chinese,Malay,Tamil
2,3,ANDERSON SECONDARY SCHOOL,4 - 10,4,10,7.0,0.870748,ANDERSONSECONDARYSCHOOL,ANDERSON SECONDARY SCHOOL,http://www.andersonsec.moe.edu.sg,...,CO-ED SCHOOL,SINGLE SESSION,SECONDARY,No,Yes,No,No,Chinese,Malay,Tamil
3,4,ANG MO KIO SECONDARY SCHOOL,11 - 16,11,16,13.5,0.482993,ANGMOKIOSECONDARYSCHOOL,ANG MO KIO SECONDARY SCHOOL,http://www.angmokiosec.moe.edu.sg,...,CO-ED SCHOOL,SINGLE SESSION,SECONDARY,No,No,No,No,Chinese,Malay,Tamil
4,,5 ANGLICAN HIGH SCHOOL,5(D) - 12(P),5,12,8.5,0.782313,ANGLICANHIGHSCHOOL,ANGLICAN HIGH SCHOOL,http://www.anglicanhigh.moe.edu.sg,...,CO-ED SCHOOL,SINGLE SESSION,SECONDARY,Yes,Yes,No,No,Chinese,na,na
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,,143 YUHUA SECONDARY SCHOOL West,14 - 22,14,22,18.0,0.064626,YUHUASECONDARYSCHOOL,YUHUA SECONDARY SCHOOL,http://www.yuhuasec.moe.edu.sg,...,CO-ED SCHOOL,SINGLE SESSION,SECONDARY,No,No,No,No,Chinese,Malay,Tamil
143,144,YUSOF ISHAK SECONDARY SCHOOL#,-,100,100,100.0,0.006803,YUSOFISHAKSECONDARYSCHOOL,YUSOF ISHAK SECONDARY SCHOOL,http://www.yusofishaksec.moe.edu.sg,...,CO-ED SCHOOL,SINGLE SESSION,SECONDARY,No,No,No,No,Chinese,Malay,Tamil
144,145,YUYING SECONDARY SCHOOL,8 - 22,8,22,15.0,0.363946,YUYINGSECONDARYSCHOOL,YUYING SECONDARY SCHOOL,http://www.yuyingsec.moe.edu.sg,...,CO-ED SCHOOL,SINGLE SESSION,SECONDARY,No,No,No,No,Chinese,Malay,Tamil
145,146,ZHENGHUA SECONDARY SCHOOL,12 - 20,12,20,16.0,0.231293,ZHENGHUASECONDARYSCHOOL,ZHENGHUA SECONDARY SCHOOL,http://www.zhenghuasec.moe.edu.sg,...,CO-ED SCHOOL,SINGLE SESSION,SECONDARY,No,No,No,No,Chinese,Malay,Tamil


In [16]:
secondary_ranked.columns

Index(['serial', 'school', 'score_range', 'score_low', 'score_high',
       'average_score', 'percentile_rank', 'match_on', 'school_name',
       'url_address', 'address', 'postal_code', 'telephone_no',
       'telephone_no_2', 'fax_no', 'fax_no_2', 'email_address', 'mrt_desc',
       'bus_desc', 'principal_name', 'first_vp_name', 'second_vp_name',
       'third_vp_name', 'fourth_vp_name', 'fifth_vp_name', 'sixth_vp_name',
       'dgp_code', 'zone_code', 'type_code', 'nature_code', 'session_code',
       'mainlevel_code', 'sap_ind', 'autonomous_ind', 'gifted_ind', 'ip_ind',
       'mothertongue1_code', 'mothertongue2_code', 'mothertongue3_code'],
      dtype='object')

In [17]:
# use geocode to get location
sch_to_geocode = secondary_ranked['school_name'] + ' Singapore '
add_to_geocode = secondary_ranked['address'] + ' Singapore ' + secondary_ranked['postal_code'].astype(str)
add_loc_cols = {'latitude':[],'longitude':[]}

for sch,add in zip(sch_to_geocode,add_to_geocode):
    # print(sch)
    # print(add)
    try:
        geo = geolocator.geocode(sch)[1]
    except TypeError:
        try:
            geo = geolocator.geocode(add)[1]
        except TypeError:
            query = building_to_data(sch)[0]
            print(query['SEARCHVAL'])
            geo = [query['LATITUDE'].replace("'",''),query['LONGITUDE'].replace("'",'')]
    # print(geo)
    add_loc_cols['latitude'].append(geo[0])
    add_loc_cols['longitude'].append(geo[1])
add_loc_cols

PAYA LEBAR METHODIST GIRLS' SCHOOL (SECONDARY)


{'latitude': [1.4459118499999999,
  1.4360948,
  1.37541705,
  1.3676706,
  1.33021825,
  1.3197093999999998,
  1.3029215,
  1.3029215,
  1.3690319,
  1.3406552,
  1.3417851,
  1.334061,
  1.3255597,
  1.3247404,
  1.3244863,
  1.3427581000000002,
  1.3708838499999998,
  1.30726495,
  1.3484109000000002,
  1.2859241,
  1.3820990499999999,
  1.34569905,
  1.45374325,
  1.35492755,
  1.35492755,
  1.33421505,
  1.33421505,
  1.3404936,
  1.3058723,
  1.3326721,
  1.39185175,
  1.37344905,
  1.37344905,
  1.27614885,
  1.4266663,
  1.3053550999999999,
  1.41910045,
  1.31566085,
  1.31969715,
  1.39652095,
  1.2934392,
  1.337598,
  1.3667425500000001,
  1.36358465,
  1.2987458,
  1.36116845,
  1.3569738500000001,
  1.4002065,
  1.44173915,
  1.30171935,
  1.3753080500000001,
  1.3473107500000001,
  1.2894694000000002,
  1.3176038499999998,
  1.3977221499999999,
  1.34467265,
  1.3745734,
  1.3533675,
  1.3716598,
  1.37776255,
  1.3520885,
  1.3262797499999999,
  1.35317525,
  1.33030995

In [18]:
secondary_ranked = secondary_ranked.assign(**add_loc_cols)
secondary_ranked

,serial,school,score_range,score_low,score_high,average_score,percentile_rank,match_on,school_name,url_address,...,mainlevel_code,sap_ind,autonomous_ind,gifted_ind,ip_ind,mothertongue1_code,mothertongue2_code,mothertongue3_code,latitude,longitude
0,1,ADMIRALTY SECONDARY SCHOOL,15 - 20,15,20,17.5,0.102041,ADMIRALTYSECONDARYSCHOOL,ADMIRALTY SECONDARY SCHOOL,http://www.admiraltysec.moe.edu.sg,...,SECONDARY,No,No,No,No,Chinese,Malay,Tamil,1.445912,103.802908
1,2,AHMAD IBRAHIM SECONDARY SCHOOL,10 - 17,10,17,13.5,0.482993,AHMADIBRAHIMSECONDARYSCHOOL,AHMAD IBRAHIM SECONDARY SCHOOL,http://www.ahmadibrahimsec.moe.edu.sg,...,SECONDARY,No,No,No,No,Chinese,Malay,Tamil,1.436095,103.830055
2,3,ANDERSON SECONDARY SCHOOL,4 - 10,4,10,7.0,0.870748,ANDERSONSECONDARYSCHOOL,ANDERSON SECONDARY SCHOOL,http://www.andersonsec.moe.edu.sg,...,SECONDARY,No,Yes,No,No,Chinese,Malay,Tamil,1.375417,103.851723
3,4,ANG MO KIO SECONDARY SCHOOL,11 - 16,11,16,13.5,0.482993,ANGMOKIOSECONDARYSCHOOL,ANG MO KIO SECONDARY SCHOOL,http://www.angmokiosec.moe.edu.sg,...,SECONDARY,No,No,No,No,Chinese,Malay,Tamil,1.367671,103.842594
4,,5 ANGLICAN HIGH SCHOOL,5(D) - 12(P),5,12,8.5,0.782313,ANGLICANHIGHSCHOOL,ANGLICAN HIGH SCHOOL,http://www.anglicanhigh.moe.edu.sg,...,SECONDARY,Yes,Yes,No,No,Chinese,na,na,1.330218,103.941759
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,,143 YUHUA SECONDARY SCHOOL West,14 - 22,14,22,18.0,0.064626,YUHUASECONDARYSCHOOL,YUHUA SECONDARY SCHOOL,http://www.yuhuasec.moe.edu.sg,...,SECONDARY,No,No,No,No,Chinese,Malay,Tamil,1.346888,103.722425
143,144,YUSOF ISHAK SECONDARY SCHOOL#,-,100,100,100.0,0.006803,YUSOFISHAKSECONDARYSCHOOL,YUSOF ISHAK SECONDARY SCHOOL,http://www.yusofishaksec.moe.edu.sg,...,SECONDARY,No,No,No,No,Chinese,Malay,Tamil,1.407173,103.896633
144,145,YUYING SECONDARY SCHOOL,8 - 22,8,22,15.0,0.363946,YUYINGSECONDARYSCHOOL,YUYING SECONDARY SCHOOL,http://www.yuyingsec.moe.edu.sg,...,SECONDARY,No,No,No,No,Chinese,Malay,Tamil,1.357094,103.890709
145,146,ZHENGHUA SECONDARY SCHOOL,12 - 20,12,20,16.0,0.231293,ZHENGHUASECONDARYSCHOOL,ZHENGHUA SECONDARY SCHOOL,http://www.zhenghuasec.moe.edu.sg,...,SECONDARY,No,No,No,No,Chinese,Malay,Tamil,1.389524,103.76531


In [19]:
secondary_ranked = gpd.GeoDataFrame(secondary_ranked, geometry=gpd.points_from_xy(secondary_ranked.longitude,secondary_ranked.latitude))
secondary_ranked

,serial,school,score_range,score_low,score_high,average_score,percentile_rank,match_on,school_name,url_address,...,sap_ind,autonomous_ind,gifted_ind,ip_ind,mothertongue1_code,mothertongue2_code,mothertongue3_code,latitude,longitude,geometry
0,1,ADMIRALTY SECONDARY SCHOOL,15 - 20,15,20,17.5,0.102041,ADMIRALTYSECONDARYSCHOOL,ADMIRALTY SECONDARY SCHOOL,http://www.admiraltysec.moe.edu.sg,...,No,No,No,No,Chinese,Malay,Tamil,1.445912,103.802908,POINT (103.80291 1.44591)
1,2,AHMAD IBRAHIM SECONDARY SCHOOL,10 - 17,10,17,13.5,0.482993,AHMADIBRAHIMSECONDARYSCHOOL,AHMAD IBRAHIM SECONDARY SCHOOL,http://www.ahmadibrahimsec.moe.edu.sg,...,No,No,No,No,Chinese,Malay,Tamil,1.436095,103.830055,POINT (103.83005 1.43609)
2,3,ANDERSON SECONDARY SCHOOL,4 - 10,4,10,7.0,0.870748,ANDERSONSECONDARYSCHOOL,ANDERSON SECONDARY SCHOOL,http://www.andersonsec.moe.edu.sg,...,No,Yes,No,No,Chinese,Malay,Tamil,1.375417,103.851723,POINT (103.85172 1.37542)
3,4,ANG MO KIO SECONDARY SCHOOL,11 - 16,11,16,13.5,0.482993,ANGMOKIOSECONDARYSCHOOL,ANG MO KIO SECONDARY SCHOOL,http://www.angmokiosec.moe.edu.sg,...,No,No,No,No,Chinese,Malay,Tamil,1.367671,103.842594,POINT (103.84259 1.36767)
4,,5 ANGLICAN HIGH SCHOOL,5(D) - 12(P),5,12,8.5,0.782313,ANGLICANHIGHSCHOOL,ANGLICAN HIGH SCHOOL,http://www.anglicanhigh.moe.edu.sg,...,Yes,Yes,No,No,Chinese,na,na,1.330218,103.941759,POINT (103.94176 1.33022)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,,143 YUHUA SECONDARY SCHOOL West,14 - 22,14,22,18.0,0.064626,YUHUASECONDARYSCHOOL,YUHUA SECONDARY SCHOOL,http://www.yuhuasec.moe.edu.sg,...,No,No,No,No,Chinese,Malay,Tamil,1.346888,103.722425,POINT (103.72243 1.34689)
143,144,YUSOF ISHAK SECONDARY SCHOOL#,-,100,100,100.0,0.006803,YUSOFISHAKSECONDARYSCHOOL,YUSOF ISHAK SECONDARY SCHOOL,http://www.yusofishaksec.moe.edu.sg,...,No,No,No,No,Chinese,Malay,Tamil,1.407173,103.896633,POINT (103.89663 1.40717)
144,145,YUYING SECONDARY SCHOOL,8 - 22,8,22,15.0,0.363946,YUYINGSECONDARYSCHOOL,YUYING SECONDARY SCHOOL,http://www.yuyingsec.moe.edu.sg,...,No,No,No,No,Chinese,Malay,Tamil,1.357094,103.890709,POINT (103.89071 1.35709)
145,146,ZHENGHUA SECONDARY SCHOOL,12 - 20,12,20,16.0,0.231293,ZHENGHUASECONDARYSCHOOL,ZHENGHUA SECONDARY SCHOOL,http://www.zhenghuasec.moe.edu.sg,...,No,No,No,No,Chinese,Malay,Tamil,1.389524,103.76531,POINT (103.76531 1.38952)


In [20]:
# declare CRS
secondary_ranked.crs = "EPSG:4326"

In [21]:
secondary_ranked.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [22]:
engine = create_engine(f'postgresql+psycopg2://{Capstone_AWS_Stuart.username}:{Capstone_AWS_Stuart.password}@{Capstone_AWS_Stuart.host}/Capstone', echo=False)

secondary_ranked.to_postgis('secondary_schools_ranked_with_loc',engine, if_exists='replace', index=False, dtype={'geometry': Geometry(geometry_type='POINT', srid= 4326)})